<a href="https://colab.research.google.com/github/yeho/scikit-learn-AI/blob/master/GA_CEDIS_tiendas_Optimizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧬 Algoritmo Genético para Optimización de Rutas de Entrega Semanal

Este notebook implementa un algoritmo genético simple para planificar entregas semanales desde un CEDIS a tiendas con frecuencias fijas usando datos sintéticos.

In [ ]:
# Configuración inicial
import random
from collections import defaultdict

DIAS_SEMANA = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]
TIENDAS = {
    "T1": {"frecuencia": 2, "demanda": 10},
    "T2": {"frecuencia": 1, "demanda": 15},
    "T3": {"frecuencia": 2, "demanda": 5},
    "T4": {"frecuencia": 1, "demanda": 8},
    "T5": {"frecuencia": 2, "demanda": 7},
}
VEHICULO_CAPACIDAD = 30
NUM_GENERACIONES = 100
POBLACION_TAM = 30
PROB_MUTACION = 0.2

In [ ]:
def generar_individuo():
    visitas = []
    for tienda, info in TIENDAS.items():
        for _ in range(info["frecuencia"]):
            dia = random.choice(DIAS_SEMANA)
            visitas.append((tienda, dia))
    random.shuffle(visitas)
    return visitas

def crear_poblacion():
    return [generar_individuo() for _ in range(POBLACION_TAM)]

def fitness(individuo):
    rutas_por_dia = defaultdict(list)
    penalizacion = 0
    for tienda, dia in individuo:
        rutas_por_dia[dia].append(tienda)
    for dia, visitas in rutas_por_dia.items():
        carga = sum(TIENDAS[t]["demanda"] for t in visitas)
        if carga > VEHICULO_CAPACIDAD:
            penalizacion += (carga - VEHICULO_CAPACIDAD) * 10
    visitas_por_tienda = defaultdict(int)
    for tienda, _ in individuo:
        visitas_por_tienda[tienda] += 1
    for tienda, info in TIENDAS.items():
        faltan = info["frecuencia"] - visitas_por_tienda[tienda]
        if faltan > 0:
            penalizacion += faltan * 50
    return 1000 - penalizacion

In [ ]:
def seleccion(poblacion):
    return sorted(poblacion, key=fitness, reverse=True)[:2]

def cruzamiento(p1, p2):
    punto = random.randint(1, len(p1) - 1)
    hijo = p1[:punto] + p2[punto:]
    return reparar_hijo(hijo)

def reparar_hijo(hijo):
    conteo_actual = defaultdict(int)
    for tienda, _ in hijo:
        conteo_actual[tienda] += 1
    for tienda, info in TIENDAS.items():
        dif = conteo_actual[tienda] - info["frecuencia"]
        if dif > 0:
            for i in reversed(range(len(hijo))):
                if hijo[i][0] == tienda and dif > 0:
                    hijo.pop(i)
                    dif -= 1
        elif dif < 0:
            for _ in range(abs(dif)):
                dia = random.choice(DIAS_SEMANA)
                hijo.append((tienda, dia))
    random.shuffle(hijo)
    return hijo

def mutacion(individuo):
    i = random.randint(0, len(individuo) - 1)
    tienda, _ = individuo[i]
    nuevo_dia = random.choice(DIAS_SEMANA)
    individuo[i] = (tienda, nuevo_dia)
    return individuo

In [ ]:
def algoritmo_genetico():
    poblacion = crear_poblacion()
    mejor = max(poblacion, key=fitness)
    for _ in range(NUM_GENERACIONES):
        nueva_poblacion = []
        elite = seleccion(poblacion)
        nueva_poblacion.extend(elite)
        while len(nueva_poblacion) < POBLACION_TAM:
            p1, p2 = random.sample(elite, 2)
            hijo = cruzamiento(p1, p2)
            if random.random() < PROB_MUTACION:
                hijo = mutacion(hijo)
            nueva_poblacion.append(hijo)
        poblacion = nueva_poblacion
        mejor_candidato = max(poblacion, key=fitness)
        if fitness(mejor_candidato) > fitness(mejor):
            mejor = mejor_candidato
    return mejor

In [ ]:
# Ejecutar
mejor_solucion = algoritmo_genetico()
print("\n📦 Mejor planificación semanal encontrada:")
for tienda, dia in sorted(mejor_solucion, key=lambda x: DIAS_SEMANA.index(x[1])):
    print(f"- {tienda} se visita el {dia}")


📦 Mejor planificación semanal encontrada:
- T5 se visita el Lunes
- T4 se visita el Miércoles
- T2 se visita el Jueves
- T1 se visita el Viernes
- T3 se visita el Viernes
- T5 se visita el Viernes
- T3 se visita el Sábado
- T1 se visita el Sábado
